![Xilinx Logo](images/xilinx_logo.png "Xilinx Logo")

# 1. Introduction

This notebook demonstrates how to capture video from a V4L2 device, processes it through filter(optional) and display the output on a monitor using a DRM/KMS display device. This notebook uses the GStreamer multimedia framework. In addition, the memory bandwidth is measured and plotted in a parallel notebook.

Three types of V4L2 devices are supported in this notebook:
* Virtual Video Test driver (vivid)
* USB Video Class (UVC) driver (usbcam)
* MIPI CSI-2 capture pipeline using the Leopard IMX274 FMC daughter card (mipi)

See nb2 for details on the mentioned V4L2 devices.

The display device uses the Xilinx DRM/KMS driver. A video mixer supports alpha blending of multiple layers (also called planes). The plane formats are fixed and configured as follows:
* 4 BG24 planes (IDs: 34-37)
* 4 YUY2 planes (IDs: 38-41)
* 1 ARGB plane (ID: 42) - this is the primary plane used for setting the CRTC resolution

The video mixer is connected to an HDMI encoder which drives the display. Both video mixer and HDMI encoder are implemented inside the FPGA.

The video pipeline is composed of the following GStreamer elements:
* The ``xlnxvideosrc`` element is used to capture video from a V4L2 device
* The ``sdxfilter2d`` element is used to implement a 2D convolution filter
* The ``perf`` element is used to measure and print the frame rate
* The ``kmssink`` element is used to display video on a monitor using the DRM/KMS kernel subsystem

The ``vmk180-trd-apm`` notebook is executed in parallel to this notebook ro measure and plot the memory bandwidth of the live video pipeline.

In this notebook, you will:
1. Create a GStreamer video pipeline that captures video from a V4L2 device and displays the video on a monitor using DRM/KMS.
2. Gstreamer pipeline with and without filter is created using the configuration parameter.
3. Run the ``trd-apm`` notebook to measure and plot the memory bandwidth while the video pipeline is running.
4. Create a GStreamer pipeline graph and view it inside this notebook.

# 2. Imports and Initialization

Import all python modules required for this notebook. 

In [ ]:
from IPython.display import Image, display, clear_output
import pydot
import sys
import time
import gi
gi.require_version('Gst', '1.0')
gi.require_version("GstApp", "1.0")
from gi.repository import GObject, GLib, Gst, GstApp

This is the VMK180 TRD notebook 3 (nb3).

In [ ]:
nb = "nb3"

Create a directory for saving the pipeline graph as dot file. Set the GStreamer debug dot directory environement variable to point to that directory.

In [ ]:
dotdir = "/home/root/gst-dot/" + nb
!mkdir -p $dotdir
%env GST_DEBUG_DUMP_DOT_DIR = $dotdir

Initialize the GStreamer library. Optionally enable debug (default off) and set the debug level.

In [ ]:
Gst.init(None)
Gst.debug_set_active(False)
Gst.debug_set_default_threshold(1)

# 3. Run the APM Notebook to Plot the Memory Bandwidth

Open the ``vmk180-trd-apm.ipynb`` notebook from the *File Browser* in a new tab. Execute the notebook by selecting *Run -> Run All Cells* from the Jupyter Lab menu bar. In section 4 of the APM notebook, a horizontal bar graph is shown that plots the currently consumed memory bandwidth split out by different AXI ports. For more information, read the APM notebook tutorial.

Once you see the graph, right-click the graph and select *Create New View for Output*. This will create a new window/tab with just the graph. Now re-arrange the window by dragging it to the the right side of the screen so it shows side-by-side with the notebook window (see screenshot below).

![APM Plot](images/apm-plot-nb3.jpg "APM Plot")

Switch tabs back to the nb3 notebook and follow the steps below. Once the video pipeline is running, you will notice the bar graph will be updated live with the measured memory bandwidth numbers in Gbps.

**Note:** You can keep the memory bandwith output view open while switching between notebooks. There is no need to restart the APM notebook.

# 4. Create and Configure the GStreamer Elements

Create the ``xlnxvideosrc`` element and set some properties:
* Set the ``io_mode`` to ``mmap`` for mapping device memory into application address space.
* Set the ``src_type`` property to the desired source device e.g. ``vivid``, ``usbcam``, or ``mipi`` (see list above)

The below code sets the default source type to ``vivid``. Update the value below next to the comment to select USB webcam or MIPI as capture device.

If MIPI is selected, change the I/O mode to DMABUF (https://www.kernel.org/doc/html/v4.16/driver-api/dma-buf.html) which allows sharing of video buffers in 0-copy fashion between the source and sink elements.

In [ ]:
src_types = ["vivid", "usbcam", "mipi"]
src_type = src_types[2] # Change the source type to vivid, usbcam, or mipi via list index

io_mode = "mmap"
if src_type == "mipi":
    io_mode = "dmabuf"

src = Gst.ElementFactory.make("xlnxvideosrc")
src.set_property("io-mode", io_mode)
src.set_property("src-type", src_type)

Create a caps filter element to set the desired resolution (width and height) and format. The caps filter is configured to parse the mentioned properties from a string.

The default resolution is set to 1280x720 and the format to YUY2 as those are commonly supported by USB webcams.

If ``mipi`` is selected as source type, the maximum supported resolution is 3840x2160 (4K) at 60 fps. Note that the connected monitor also needs to support this resolution, otherwise the pipeline will fail during caps negotiation (see modeprint output below).

In [ ]:
res_dict = {
    "720p" : ("1280", "720"),
    "1080p" : ("1920", "1080"),
    "2160p" : ("3840", "2160")
}
res = "2160p" # Change the resolution string to 720p, 1080p, or 2160p (mipi only)
width = res_dict[res][0]
height = res_dict[res][1]
print("Selected resolution: " + width + "x" + height)

fmt = "YUY2"

caps = Gst.ElementFactory.make("capsfilter")
cap = Gst.Caps.from_string("video/x-raw, width=" + str(width) + ", height=" + str(height) + ", format=" + fmt)
caps.set_property("caps", cap)

In [ ]:
add_filter = '0' # By default pipeline is created without filter, change this variable to 1 to add filter.

Set XRT related environment variables.

In [ ]:
if add_filter == '1':
    %env XILINX_XRT = /usr
    %env XCL_BINDIR = /media/sd-mmcblk0p1

Create the sdxfilter2d element. The filter2d element has a couple properties to specify the implementation of the kernel:

The filter-mode property determines whether the kernel is run in
software ("SW") on PS or
accelerated in hardware ("HW").
The filter-kernel is set to filter2d_pl_accel property if "HW" mode is selected:

In the below cell, change the filter_mode variable to select the filter mode as per above description.

**Note:** Filter element will be created only if ``add_filter``is set to 1

In [ ]:
if add_filter == '1':
    filter_modes = ["HW", "SW"]
    filter_mode = filter_modes[0] # Change filter mode to HW or SW via list index
    print("Selected filter mode: " + filter_mode)

    filter_kernel = "filter2d_pl_accel"

    filter2d = Gst.ElementFactory.make("sdxfilter2d")
    filter2d.set_property("filter-mode", filter_mode)
    filter2d.set_property("filter-kernel", filter_kernel)

The PL filter2d has two ways of programming the filter coefficients:

by setting the filter-preset property
by setting the coefficients property
First, we look at how to set the filter-preset property. The below command returns a list of supported filter-presets to choose from.

In [ ]:
!gst-inspect-1.0 sdxfilter2d | grep -A 13 filter-preset

Based on the above list, set the filter_preset variable to one of the string values listed on the left e.g. "horizontal sobel" which in turn is used to set the corresponding element property. The preset implicitly programs the filter coefficients behind the scene. When reading back the property value, the enum value is returned which is listed on the right e.g. "GST_SDXFILTER2D_PRESET_HSOBEL".

**Note:** Filter property is set only if ``add_filter``is set to 1

In [ ]:
if add_filter == '1':
    filter_preset = "horizontal sobel" # Change the filter preset to any of the pre-defined values

    filter2d.set_property("filter-preset", filter_preset)
    print("filter-preset: " + str(filter2d.get_property("filter-preset")))

The second way is to explicitly program the filter coefficients via the coefficients property. Setting the coefficients this way will override the preset, hence the below code is commented out by default. If you want to set the coefficients by value rather than by preset, uncomment the below code.

The filter coefficients are a 3x3 matrix of short int values. The default values in the below code snippet correspond to the identity matrix which results is a simple passthrough. The identity coefficients are as follows:

 0  0  0
 0  1  0
 0  0  0
To match the coefficients for "horizontal sobel", use the following matrix:

 1  2  1
 0  0  0
-1 -2 -1

**Note:** Filter coefficients are set only if ``add_filter``is set to 1

In [ ]:
if add_filter == '1':
    def print_coeff(coeff):
        print("coefficients: ")
        for i in range(0, 3):
            for j in range(0, 3):
                print(str(coeff[i][j]) + ' ', end = '')
            print(' ')

# Uncomment the below lines to use user-defined coefficient values instead of presets
#coeff = Gst.ValueArray([Gst.ValueArray([0, 0, 0]), Gst.ValueArray([0, 1, 0]), Gst.ValueArray([0, 0, 0])]) # identity coefficients
#coeff = Gst.ValueArray([Gst.ValueArray([1, 2, 1]), Gst.ValueArray([0, 0, 0]), Gst.ValueArray([-1, -2, -1])]) # horizontal sobel coefficients
#filter2d.set_property("coefficients", coeff)
#new_coeff = filter2d.get_property("coefficients")
#print_coeff(new_coeff)

Create the ``perf`` element which is used to measure and print the frame rate while the video pipeline is running.

In [ ]:
perf = Gst.ElementFactory.make("perf")

The display driver creates a DRM device node with the module name ``xlnx``.

List information about the DRM device by passing the module name to the ``modeprint`` utility.

In [ ]:
!modeprint xlnx

Create the ``kmssink`` element and set some properties:
* Set the ``driver-name`` property to the Xilinx DRM driver name ``xlnx``.
* Set the ``plane-id`` property to the ID value of the target plane. The default value 34 is set to the first YUY2 plane.
* Set the ``fullscreen-overlay`` property to ``False`` to keep the CRTC set to the native display resolution.
* Set the ``render-rectangle`` property to a quadruple consisting of x-offset, y-offset, width, and height. The render-rectangle allows moving a plane position on the display.

In [ ]:
driver_name = "xlnx"
plane_id = 38
xoff = 0 # Change this value to move the plane position in the x-direction
yoff = 0 # Change this value to move the plane position in the y-direction
width = int(width, 10)
height = int(height, 10)
render_rectangle = Gst.ValueArray((xoff, yoff, width, height))

sink = Gst.ElementFactory.make("kmssink")
sink.set_property("driver-name", driver_name)
sink.set_property("plane-id", plane_id)
sink.set_property("render-rectangle", render_rectangle)
sink.set_property("sync", False)

# Uncomment the below code to read back the newly set property values
#print("sink properties: ")
#print("driver-name: " + str(sink.get_property("driver-name")))
#print("plane-id: " + str(sink.get_property("plane-id")))


# 5. Create and Run the GStreamer Pipeline

Create the pipeline, add all elements, and link them together.

In [ ]:
pipeline = Gst.Pipeline.new(nb)

pipeline.add(src)
pipeline.add(caps)
pipeline.add(perf)
if add_filter == '1':
    pipeline.add(filter2d) # Create the filter2d element
pipeline.add(sink)

src.link(caps)
if add_filter == '1':
    caps.link(filter2d) # Link the fiter2d element's sink pad to the caps element 
    filter2d.link(perf)
else:
    caps.link(perf)
perf.link(sink);
print(pipeline)

The ``bus_call`` function listens on the bus for ``EOS``, ``INFO`` and ``ERROR`` events. In case of ``EOS`` or ``ERROR``, stop the pipeline (set to ``NULL`` state) and quit the main loop. 

For ``INFO`` and ``ERROR`` events, parse and print the info/error message. The ``perf`` element generates ``INFO`` events with the measured frame rate.

In [ ]:
def bus_call(bus, message, loop):
    t = message.type
    if t == Gst.MessageType.EOS:
        sys.stdout.write("End-of-stream\n")
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    elif t == Gst.MessageType.INFO:
        err, info = message.parse_info()
        sys.stderr.write("Info: %s\n" % info)
        clear_output(wait=True)
    elif t == Gst.MessageType.ERROR:
        err, debug = message.parse_error()
        sys.stderr.write("Error: %s: %s\n" % (err, debug))
        pipeline.set_state(Gst.State.NULL)
        loop.quit()
    return True

Start the pipeline (set to ``PLAYING`` state), create the main loop and listen to messages on the bus. Register the ``bus_call`` callback function with the ``message`` signal of the bus. Start the main loop.

The video will be displayed on the monitor. The frame rate will be printed and updated below the code cell.

To stop the pipeline, click the square shaped icon labelled 'Interrupt the kernel' in the top menu bar. Create a dot graph of the pipeline topology before stopping the pipeline. Quit the main loop.

In [ ]:
pipeline.set_state(Gst.State.PLAYING);

loop = GLib.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect("message", bus_call, loop)

try:
    loop.run()
except:
    sys.stdout.write("Interrupt caught\n")
    Gst.debug_bin_to_dot_file(pipeline, Gst.DebugGraphDetails.ALL, nb)
    pipeline.set_state(Gst.State.NULL)
    loop.quit()
    pass

# 7. Summary

In this notebook you learned how to:
1. Create a GStreamer pipeline that demonstrates how to capture video from a V4L2 device and display it on a monitor
2. Plot the live memory bandwidth by running the APM notebook in parallel
3. Export the pipeline topology as a dot file image and display it in the notebook

<center>Copyright© 2019 Xilinx</center>